In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report


df_aston = pd.read_excel('aston_origin.xlsx')
df_andersen = pd.read_excel('andersen_origin.xlsx')

In [2]:
del df_aston['course_link']
del df_aston['course_modules_content']
del df_aston['course_module_names']
del df_andersen['course_link']
del df_andersen['course_modules_content']
del df_andersen['course_module_names']
del df_andersen['proficiency_level']
#лишние для обучения поля

In [3]:
#удаляем строки с пустыми или слишком короткими значениями

for df in [df_andersen, df_aston]:
    df.drop(
        df[(df['summary'].str.len() < 5) | (df['course_structure'].str.len() < 5)].index,
        inplace=True
    )

In [4]:
df_andersen['recommended_grade'].unique() 
# зоопарк из различных грейдов

array(['M1+', 'J2', 'J1+', nan, 'J3+', 'J3', 'J2+', 'J1−J3', 'J1',
       'All Grades', 'M2', 'М2', 'M1', 'М1+', 'M2+', 'М3', 'J1-J2',
       'J1−J2', 'All grades', 'М1', 'M3', 'S1', 'Trainee', 'j3+', 'М3+',
       'М2+', 'J3-M1', 'J3−М1', 'All Grade', 'S1+', 'M3+', 'S2+', 'J1-J3',
       'Trainee, J1', 'All gardes', 'О1+'], dtype=object)

In [5]:
def normalize_grade(val):
    if pd.isna(val):
        return np.nan
    
    s = str(val).strip().lower()
    
    # Замена кириллицы на латиницу
    s = s.replace('м', 'm').replace('о', 'o')
    
    # Исправляем варианты "All grades"
    if 'all' in s:
        return 'All'
    if 'Trainee' in s:
        return 'J'
    
    # Если курс предназначен для джуна любого уровня, он переходит в класс 'J'
    # Если курс предназначен, например, для 'J3−М1', он тоже переходит в J
    if 'j' in s:
        return 'J'
    elif 'm' in s:
        return 'M'
    elif 's' in s:
        return 'S'
    
    return np.nan

df_andersen['recommended_grade'] = df_andersen['recommended_grade'].apply(normalize_grade)

In [6]:
df_andersen['recommended_grade'].value_counts()

recommended_grade
J      764
M      249
All    216
S       22
Name: count, dtype: int64

In [7]:
def clean_text(text):

    """Удаляет HTML, убирает лишние пробелы, переводит в нижний регистр"""
    
    if pd.isna(text):
        return text
    text = BeautifulSoup(text, "html.parser").get_text(" ")
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()

def clean_dataset(df):
    df = df.copy()
    
    #чистим текстовые поля
    df["summary"] = df["summary"].apply(clean_text)
    df["course_structure"] = df["course_structure"].apply(clean_text)
    df["course_name"] = df["course_name"].apply(lambda x: x.strip().lower() if pd.notna(x) else x)
    df["category_name"] = df["category_name"].apply(lambda x: x.strip().title() if pd.notna(x) else x)

    # Преобразуем duration_hours в число
    df["duration_hours"] = pd.to_numeric(df["duration_hours"], errors="coerce")
    
    return df

df_andersen_clean = clean_dataset(df_andersen)
df_aston_clean = clean_dataset(df_aston)

In [8]:
# df_andersen_clean.to_csv("df_andersen_clean_for_LLM.csv", index=False, encoding='UTF-8')
# файл, который будет использоваться для GPT

In [9]:
df_s = df_andersen_clean.loc[df_andersen_clean['recommended_grade'] == 'S', ['recommended_grade', 'course_id', 'course_name']]

In [10]:
# Найти совпадающие строки по course_name и duration_hours
matching_rows = pd.merge(
    df_aston,
    df_andersen,
    on=['course_name', 'duration_hours', 'category_name'],
    how='inner'
)

In [11]:
'''Заполняем значения recommended_grade в aston для совпадающих курсов из andersen'''

# Сначала делаем merge, чтобы получить колонку с grade из df_andersen
df_aston = df_aston.merge(
    df_andersen[['course_name', 'duration_hours', 'recommended_grade']],
    on=['course_name', 'duration_hours'],
    how='left',
    suffixes=('', '_andersen')
)

# Заполняем пропуски в df_aston['recommended_grade'] значениями из df_andersen
df_aston['recommended_grade'] = df_aston['recommended_grade'].fillna(df_aston['recommended_grade_andersen'])

# Убираем временную колонку
df_aston = df_aston.drop(columns=['recommended_grade_andersen'])

# Проверим результат
print(df_aston[['course_name', 'duration_hours', 'recommended_grade']].head())

                                         course_name duration_hours  \
0  Тренинг проведения собеседований для Рекрутеро...              4   
1                                Основы Тестирования             28   
2                                           HR Talks             16   
3                                Intro to SalesForce              3   
4                          JS online (Odessa, 19-20)             24   

  recommended_grade  
0               NaN  
1               NaN  
2                 J  
3                 J  
4                 J  


In [12]:
# проверка пригодности полей

def check_data_quality(df, target_col):
        # Кол-во уникальных значений по категориальным признакам
    print("\n=== Уникальные значения по признакам ===")
    for col in ["course_name", "summary", "category_name", "course_structure"]:
        print(f"{col}: {df[col].nunique()} уникальных")

    for df in [df_andersen_clean, df_aston_clean]:
        df["summary_len"] = df["summary"].apply(lambda x: len(str(x).split()))
        df["structure_len"] = df["course_structure"].apply(lambda x: len(str(x).split()))

In [13]:

# baseline

def train_and_predict(df_train, df_test):
    features = ["course_name", "summary", "category_name", "duration_hours", "course_structure"]
    target = "recommended_grade"

    df_train = df_train.copy()
    df_test = df_test.copy()

    # Убираем строки без метки
    df_train = df_train.dropna(subset=[target])

    # Заполняем пропуски в текстах
    for col in ["course_name", "summary", "category_name", "course_structure"]:
        df_train[col] = df_train[col].fillna("")
        df_test[col] = df_test[col].fillna("")

    # duration_hours — приведение к числу и заполнение медианой
    df_train["duration_hours"] = pd.to_numeric(df_train["duration_hours"], errors="coerce")
    df_test["duration_hours"] = pd.to_numeric(df_test["duration_hours"], errors="coerce")
    median_val = df_train["duration_hours"].median()
    df_train["duration_hours"] = df_train["duration_hours"].fillna(median_val)
    df_test["duration_hours"] = df_test["duration_hours"].fillna(median_val)

    # Разделяем train/val
    X_train, X_val, y_train, y_val = train_test_split(
        df_train[features], df_train[target], test_size=0.2, random_state=42, stratify=df_train[target]
    )

    # Преобразования
    preprocessor = ColumnTransformer(
        transformers=[
            ("summary", TfidfVectorizer(max_features=5000, ngram_range=(1, 2)), "summary"),
            ("course_name", TfidfVectorizer(max_features=1000), "course_name"),
            ("course_structure", TfidfVectorizer(max_features=2000), "course_structure"),
            ("cat", OneHotEncoder(handle_unknown="ignore"), ["category_name"]),
            ("num", StandardScaler(), ["duration_hours"])
        ],
        remainder="drop"
    )

    model = Pipeline([
        ("preprocessor", preprocessor),
        ("clf", LGBMClassifier(n_estimators=500,
    learning_rate=0.05,
    num_leaves=64,
    colsample_bytree=0.8,
    subsample=0.8,
    random_state=42))
    ])

    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    print("\n=== Отчет по валидации ===")
    print(classification_report(y_val, y_pred))

    df_test[target] = model.predict(df_test[features])

    return df_test, model

In [14]:
check_data_quality(df_andersen_clean, "recommended_grade")

df_aston_with_preds, trained_model = train_and_predict(df_andersen_clean, df_aston_clean)

# df_aston_with_preds.to_excel("df_aston_with_preds_without_balance.xlsx", index=False)


=== Уникальные значения по признакам ===
course_name: 1346 уникальных
summary: 1273 уникальных
category_name: 53 уникальных
course_structure: 1334 уникальных
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007244 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12731
[LightGBM] [Info] Number of data points in the train set: 1000, number of used features: 579
[LightGBM] [Info] Start training from score -1.754464
[LightGBM] [Info] Start training from score -0.492658
[LightGBM] [Info] Start training from score -1.614450
[LightGBM] [Info] Start training from score -4.074542
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spl

c:\Users\Ideapad\OneDrive\Desktop\Grades\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\Ideapad\OneDrive\Desktop\Grades\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Причины:
1. Слишком много уникальных категориальных признаков
2. Переизбыток признаков
3. Гиперпараметры такое себе
4. LabelEncoder тоже плохая идея

Но accuracy около 70%, что радует 

In [15]:
#df_aston_with_preds['recommended_grade'].value_counts()

In [16]:
#df_andersen_clean['recommended_grade'].value_counts()